In [1]:
import json
import base64
from pydub import AudioSegment
import io
import numpy as np
from IPython.display import JSON, Audio

In [21]:
def normalizar(x, secs_ventana=2, fs=44100):
    x2 = x.copy().astype(float)
    samples_ventana = int(secs_ventana*fs)
    aux = max(np.abs(x[:samples_ventana]).max(), 1)
    x2[:samples_ventana] = x2[:samples_ventana] / aux
    
    for i in range(samples_ventana, len(x2)):
        aux = max(np.abs(x[(i+1-samples_ventana):(i+1)]).max(), 1)
        x2[i] = x2[i] / aux
    return x2

In [3]:
with open('./data/ml_data.json') as f:
    data = json.load(f)

In [24]:
sr = 22500
target_sr = 22500

aux = base64.b64decode(data[1]['audio']['data'].encode('utf-8'))
ext = data[0]["audio"]["s3"].split(".")[-1]
sound = AudioSegment.from_file(io.BytesIO(aux), format=ext).set_channels(1).set_frame_rate(target_sr)
sound = sound.compress_dynamic_range(threshold=-40)
#sound = sound.normalize()

num = np.array(sound.get_array_of_samples())
sr = sound.frame_rate
num_norm = normalizar(num, secs_ventana=2, fs=sr)

display(Audio(num_norm, rate=sr))

In [33]:
num_norm.shape

(499200,)

In [25]:
from scipy.io.wavfile import write

#data = np.random.uniform(-1,1,44100) # 44100 random samples between -1 and 1
scaled = np.int16(num_norm/np.max(np.abs(num_norm)) * sr)
write('test.wav', 22500, scaled)

display(Audio('test.wav'))

In [36]:
import base64

norm_audios = []

for item in data:
    sr = 22500
    target_sr = 22500

    aux = base64.b64decode(item['audio']['data'].encode('utf-8'))
    ext = data[0]["audio"]["s3"].split(".")[-1]
    sound = AudioSegment.from_file(io.BytesIO(aux), format=ext).set_channels(1).set_frame_rate(target_sr)
    sound = sound.compress_dynamic_range(threshold=-40)
    #sound = sound.normalize()

    num = np.array(sound.get_array_of_samples())
    sr = sound.frame_rate
    num_norm = normalizar(num, secs_ventana=2, fs=sr)
    
    scaled = np.int16(num_norm/np.max(np.abs(num_norm)) * sr)
    write('/tmp/test.wav', 22500, scaled)
    
    with open('/tmp/test.wav', 'rb') as fd:
        audio = fd.read()
        
    norm_audios.append({
        "id": item['id'],
        "audio": base64.b64encode(audio).decode('utf-8')
    })

In [37]:
import json

with open('./norm_data.json', 'w') as f:
    f.write(json.dumps(norm_audios))